In [1]:
import re
import json
import argparse
import openai
import configparser
import os


In [23]:

def get_config_apikey(cfg_file):
    config = configparser.ConfigParser()
    config.read(cfg_file)
    api_key = config.get('Config', 'openai-apikey', fallback=None)
    return api_key

def get_config_code(cfg_file):
    config = configparser.ConfigParser()
    config.read(cfg_file)
    code = config.get('Config', 'code_language', fallback=None)
    return code

def get_config_prompt(cfg_file):
    config = configparser.ConfigParser()
    config.read(cfg_file)
    prompt = config.get('Config', 'codeMaster_prompt', fallback=None)
    return prompt

api_key = get_config_apikey('./.cfg')
master_prompt = get_config_prompt('./.cfg')
openai.api_key = api_key
print(master_prompt)
def find_segments_c(text):
    segments = []
    stack = []
    start = 0
    for i, char in enumerate(text):
        if char == '{':
            if i > start:
                # save the code segment before this {
                segments.append(text[start:i].strip())
            stack.append(i)
            start = i
        elif char == '}':
            stack.pop()
            if not stack:
                # save this function
                segments.append(text[start:i+1].strip())
                start = i+1
    # save the last code segment
    if start < len(text):
        segments.append(text[start:].strip())
    return segments

def print_json(filename):
    with open(filename, 'r') as file:
        data = json.load(file)
    for segment in data:
        print(segment)
        print()



    
def askGpt(*args):
    prompt = ' '.join(args)
    response = openai.Completion.create(
        model="text-davinci-003",
        prompt=master_prompt+prompt,
        temperature=0,
        max_tokens=64,
        top_p=1.0,
        frequency_penalty=0.0,
        presence_penalty=0.0,
        stop=["\"\"\""]
    )
    generated_text = response['choices'][0]['text'].strip()
    #print(generated_text)
    return generated_text





"這是什麼，請以最短的字數幫我做標注"


In [27]:
def process_file(input_file):
    output_filename = 'codingMaster_' + input_file

    # 根據副檔名決定註解的前綴
    if input_file.endswith(('.cpp', '.c', '.ino')):
        comment_prefix = '//'
    elif input_file.endswith('.py'):
        comment_prefix = '#'
    else:
        raise ValueError(f'Unsupported file type: {input_file}')  # 如果不支援該副檔名，則拋出異常

    with open(input_file, 'r') as infile, open(output_filename, 'w') as outfile:
        for line in infile:
            stripped_line = line.strip()
            if not stripped_line:  # 如果是空白行（只有空格、tab或換行符），則跳過此次迴圈
                outfile.write(line)  # 寫入空行
                continue
            if comment_prefix in stripped_line or '{' in stripped_line or '}' in stripped_line:  # 如果該行已經包含註解、"{" 或 "}"
                outfile.write(line)  # 寫入原始行
                continue
            print(line)
            processed_line = askGpt(stripped_line)  # 對每一行調用 askGpt 函數
            outfile.write(line.rstrip('\n') + ' ' + comment_prefix + ' ' + processed_line + '\n')  # 將處理後的結果追加到該行並寫入新檔案

    print(f'Processed file has been saved to: {output_filename}')


In [ ]:
#input_file = 'arduino.ino'  

#input_file = 'python_demo.py'
input_file = 'training.py'

process_file(input_file)


import torch

torch.backends.cudnn.deterministic = True

import argparse

import numpy as np

import os

import shutil

import time

import queue

from signjoey.model import build_model

from signjoey.batch import Batch

from signjoey.helpers import (

    log_data_info,

    load_config,

    log_cfg,

    load_checkpoint,

    make_model_dir,

    make_logger,

    set_seed,

